In [507]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tweepy
import json
import re

## Gather

In [508]:
# twitter scrapper

consumer_key = 'HYKRjVWmWKjN4GsOwZKLJDShj'
consumer_secret = 'Ra0IBwR1hyP6jy0kA67xdVD2JQCS24sXydYmln435hbxhkRzxz'
access_token = '368994810-w5W5AvEZbaHKB2vLqFLoVsYPyTIGzr9dqAmbmlxR'
access_secret = 'PVZtjBuIUcObj420aNSOFmTMaJx1JNnQDe3yf5lzq2eNu'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [509]:
# Ingest Data

dog_predictions = pd.read_csv("https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv", sep='\t')
dog_data = pd.read_csv("twitter-archive-enhanced.csv")

In [510]:
# List of tweets to scrape

tweets = dog_predictions["tweet_id"]

In [511]:
# Scraper (turned off)

'''array = []
with open("tweet_json.txt", "w+") as file:
    for tweet in tweets:
        try:
            array.append(api.get_status(tweet, tweet_mode="extended")._json)
        except tweepy.TweepError as te:
            print("failed")
    json.dump(array, file, indent=True)'''

'array = []\nwith open("tweet_json.txt", "w+") as file:\n    for tweet in tweets:\n        try:\n            array.append(api.get_status(tweet, tweet_mode="extended")._json)\n        except tweepy.TweepError as te:\n            print("failed")\n    json.dump(array, file, indent=True)'

In [512]:
# Load the data from the text document

data = json.load(open("tweet_json.txt", 'r'))        

In [513]:
# Create a dataframe from the twitter data

df_list = []
for json in data:
    df_list.append({"tweet_id": json["id"],
                    "retweets": json["retweet_count"], 
                    "favorites": json["favorite_count"]})
    
tweet_data = pd.DataFrame(df_list, columns=['tweet_id', 'retweets', 'favorites'])

## Assess

In [514]:
tweet_data.head()

,tweet_id,retweets,favorites
0,666020888022790149,517,2565
1,666029285002620928,47,130
2,666033412701032449,44,125
3,666044226329800704,141,298
4,666049248165822465,41,109


In [515]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 3 columns):
tweet_id     2068 non-null int64
retweets     2068 non-null int64
favorites    2068 non-null int64
dtypes: int64(3)
memory usage: 48.5 KB


2068 rows with 3 columns

Lots of missing data. Let us examine this

In [516]:
dog_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [517]:
dog_data[dog_data['in_reply_to_status_id'].notnull()]["text"]

30      @NonWhiteHat @MayhewMayhem omg hello tanner yo...
55      @roushfenway These are good dogs but 17/10 is ...
64                        @RealKentMurphy 14/10 confirmed
113     @ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...
148     @Jack_Septic_Eye I'd need a few more pics to p...
149     Ladies and gentlemen... I found Pipsy. He may ...
179                          @Marc_IRL pixelated af 12/10
184     THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...
186     @xianmcguire @Jenna_Marbles Kardashians wouldn...
188     @dhmontgomery We also gave snoop dogg a 420/10...
189     @s8n You tried very hard to portray this good ...
218     @markhoppus MARK THAT DOG HAS SEEN AND EXPERIE...
228     Jerry just apuppologized to me. He said there ...
234     .@breaannanicolee PUPDATE: Cannon has a heart ...
251     PUPDATE: I'm proud to announce that Toby is 23...
274     @0_kelvin_0 &gt;10/10 is reserved for puppos s...
290                                    @markhoppus 182/10
291     @bragg

I'm going to drop the following columns from dog_data

- in_reply_to_status_id
- in_reply_to_user_id
- retweeted_status_id
- retweeted_status_user_id
- retweeted_status_timestamp

I also need to remove the html from source

I will also remove the ratings and img url from link because I already have them elsewhere

I will make a separate hashtag column

In [518]:
dog_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


I'm going to rename the p1, p2, p3 to make it clearer those are the most likely dogs

- prediction_1
- prediction_2
- prediction_3

I'm going to rename the p1_dog, p2_dog, p3_dog to make it clearer they are predicting dogs or not

- predicted_dog_1
- predicted_dog_2
- predicted_dog_3

In [519]:
# check for duplicates

dog_data[dog_data.duplicated()]
dog_predictions[dog_predictions.duplicated()]
tweet_data[tweet_data.duplicated()]

,tweet_id,retweets,favorites


no duplicates

In [520]:
dog_data[dog_data["retweeted_status_user_id"].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


Actually, it turns out retweets are duplicates

I will remove the retweets

In [521]:
dog_data["source"].unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

I'm going to use the beautifulsoup library to get the stuff inside the a tag

## Clean

In [522]:
# create copies

dog_data_clean = dog_data.copy()
dog_predictions_clean = dog_predictions.copy()
tweet_data_clean = tweet_data.copy()

In [523]:
# remove retweets

dog_data_clean = dog_data_clean[dog_data_clean["retweeted_status_user_id"].isnull()]

In [524]:
# drop columns

dog_data_clean.drop(["in_reply_to_status_id", "in_reply_to_user_id", "retweeted_status_id", "retweeted_status_user_id", "retweeted_status_timestamp"], axis=1, inplace=True)

In [525]:
# create hashtags column

dog_data_clean["hashtags"] = dog_data_clean["text"].apply(lambda x: re.findall(r'[#]\w+', x))
dog_data_clean["hashtags"] = dog_data_clean["hashtags"].apply(lambda x: ", ".join(x))

In [526]:
# remove ratings and url from text

dog_data_clean["text"] = dog_data_clean["text"].apply(lambda x:x.split('.')[:-2])
dog_data_clean["text"] = dog_data_clean["text"].apply(lambda x:",".join(x))

In [531]:
# Add none for empty hashtags

def fill_hashtags(data):
    if data["hashtags"] == "":
        return None
    else:
        return data["hashtags"]
    
dog_data_clean['hashtags'] = dog_data_clean.apply(fill_hashtags, axis=1)

In [528]:
# Remove html from source

dog_data_clean["source"] = dog_data_clean["source"].apply(lambda x: BeautifulSoup(x, "lxml").get_text())

In [533]:
dog_data_clean

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,hashtags
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,"This is Phineas, He's a mystical boy, Only eve...",https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,None
1,892177421306343426,2017-08-01 00:17:27 +0000,Twitter for iPhone,"This is Tilly, She's just checking pup on you,...",https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,None
2,891815181378084864,2017-07-31 00:18:03 +0000,Twitter for iPhone,"This is Archie, He is a rare Norwegian Pouncin...",https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,None
3,891689557279858688,2017-07-30 15:58:51 +0000,Twitter for iPhone,"This is Darla, She commenced a snooze mid meal",https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,None
4,891327558926688256,2017-07-29 16:00:24 +0000,Twitter for iPhone,"This is Franklin, He would like you to stop ca...",https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,#BarkWeek
5,891087950875897856,2017-07-29 00:08:17 +0000,Twitter for iPhone,Here we have a majestic great white breaching ...,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None,#BarkWeek
6,890971913173991426,2017-07-28 16:27:12 +0000,Twitter for iPhone,"Meet Jax, He enjoys ice cream so much he gets ...","https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None,None
7,890729181411237888,2017-07-28 00:22:40 +0000,Twitter for iPhone,When you watch your owner call another dog a g...,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None,None
8,890609185150312448,2017-07-27 16:25:51 +0000,Twitter for iPhone,"This is Zoey, She doesn't want to be one of th...",https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None,#BarkWeek
9,890240255349198849,2017-07-26 15:59:51 +0000,Twitter for iPhone,"This is Cassie, She is a college pup, Studying...",https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None,None


In [538]:
# rename dog prediction columns

dog_predictions_clean = dog_predictions_clean.rename(columns = {"p1": "top_prediction", "p1_conf": "top_prediction_confidence", "p1_dog":"top_prediction_is_dog", "p2": "second_prediction", "p2_conf": "second_prediction_confidence", "p2_dog":"second_prediction_is_dog", "p3": "third_prediction", "p3_conf": "third_prediction_confidence", "p3_dog":"third_prediction_is_dog"})

In [540]:
# remove image num

dog_predictions_clean.drop("img_num", axis=1, inplace=True)

In [547]:
dog_predictions_clean[dog_predictions_clean["top_prediction_is_dog"]==False]["top_prediction"].value_counts()

seat_belt                   22
web_site                    19
teddy                       18
tennis_ball                  9
dingo                        9
doormat                      8
tub                          7
swing                        7
Siamese_cat                  7
bath_towel                   7
hamster                      7
llama                        6
home_theater                 6
ice_bear                     6
car_mirror                   6
shopping_cart                5
minivan                      5
hippopotamus                 5
ox                           5
porcupine                    5
jigsaw_puzzle                4
goose                        4
hog                          4
patio                        4
bathtub                      4
guinea_pig                   4
wombat                       4
barrow                       4
Arctic_fox                   4
brown_bear                   4
                            ..
tiger_shark                  1
hand_blo

In [548]:
dog_predictions_clean[dog_predictions_clean["top_prediction"]=="seat_belt"]

,tweet_id,jpg_url,top_prediction,top_prediction_confidence,top_prediction_is_dog,second_prediction,second_prediction_confidence,second_prediction_is_dog,third_prediction,third_prediction_confidence,third_prediction_is_dog
43,666776908487630848,https://pbs.twimg.com/media/CUDeDoWUYAAD-EM.jpg,seat_belt,0.375057,False,miniature_pinscher,0.167175,True,Chihuahua,0.086951,True
108,667878741721415682,https://pbs.twimg.com/media/CUTILFiWcAE8Rle.jpg,seat_belt,0.200373,False,miniature_pinscher,0.106003,True,schipperke,0.104733,True
198,669625907762618368,https://pbs.twimg.com/media/CUr9NjgU8AEpf5w.jpg,seat_belt,0.874502,False,golden_retriever,0.055408,True,Labrador_retriever,0.026854,True
235,670427002554466305,https://pbs.twimg.com/media/CU3VzVwWwAAAsst.jpg,seat_belt,0.952258,False,toy_terrier,0.038872,True,beagle,0.003226,True
522,676582956622721024,https://pbs.twimg.com/media/CWO0m8tUwAAB901.jpg,seat_belt,0.790028,False,Boston_bull,0.196307,True,French_bulldog,0.012429,True
551,677557565589463040,https://pbs.twimg.com/media/CWcrAVQWEAA6QMp.jpg,seat_belt,0.277257,False,Shih-Tzu,0.249017,True,Pekinese,0.209213,True
577,678740035362037760,https://pbs.twimg.com/media/CWtede2WIAAF_AJ.jpg,seat_belt,0.787164,False,sunglasses,0.045739,False,beagle,0.022525,True
642,681339448655802368,https://pbs.twimg.com/media/CXSanNkWkAAqR9M.jpg,seat_belt,0.532441,False,Labrador_retriever,0.094615,True,kuvasz,0.089863,True
657,682303737705140231,https://pbs.twimg.com/media/CXgHoLnWAAA8i52.jpg,seat_belt,0.997659,False,Lakeland_terrier,0.001731,True,Airedale,0.000204,True
740,687312378585812992,https://pbs.twimg.com/media/CYnS9VWW8AAeR8m.jpg,seat_belt,0.703561,False,Great_Dane,0.139909,True,Weimaraner,0.021112,True
